In [1]:
import numpy as np
import pandas as pd
from biosppy.signals import ecg

* wavelate (+++++)
    * cD_list,cA_list (sym6)
* U_list,U_value_list (++++)
* fft (++++)
* wave indices list (++++)
    * T_list,S_list,R_list,Q_list,P_list,ST_list,QRS_list,PR_list,QRS_T_list,QRS_P_list
* correlation (+++??)
* ptp (largest - smallest) (+++)
* energy list (++)

In [9]:
def mean_squared_distance(rpeaks):
    diff = np.diff(rpeaks)
    dist = np.mean(diff*diff)
    return dist

def get_feature_list(feature, axis=-1):
    feats = np.array([])
    
    if axis == -1:
        feats = np.append(feats, np.mean(feature))
        feats = np.append(feats, np.median(feature))
        feats = np.append(feats, np.min(feature))
        feats = np.append(feats, np.max(feature))
        feats = np.append(feats, np.std(feature))
    elif axis == 0:
        feats = np.append(feats, np.mean(feature, axis = 0))
        feats = np.append(feats, np.median(feature, axis = 0))
        feats = np.append(feats, np.min(feature, axis = 0))
        feats = np.append(feats, np.max(feature, axis = 0))
        feats = np.append(feats, np.std(feature, axis = 0))
        
    return feats

def extract_feature(X_signal, sampling_rate):
    ts, filtered, rpeaks, templates_ts, templates, heart_rate_ts, heart_rate = ecg.ecg(X_signal, sampling_rate, show = False)
    rpeaks = ecg.correct_rpeaks(signal = X_signal, rpeaks = rpeaks, sampling_rate = sampling_rate, tol = 0.01)  
    peak_values = X_signal[rpeaks]
    
    print(templates.shape)
    feats = np.array([])
    
    feats = np.concatenate([feats, get_feature_list(peak_values)])
    feats = np.concatenate([feats, get_feature_list(rpeaks)])
    feats = np.append(feats, np.sqrt(mean_squared_distance(rpeaks)))
    feats = np.concatenate([feats, get_feature_list(np.diff(rpeaks))])
    
    feats = np.concatenate([feats, get_feature_list(templates, axis = 0)])
    
    
    heart_rate = np.array([np.nan, np.nan])  if len(heart_rate) == 0 else heart_rate
    feats = np.concatenate([feats, get_feature_list(heart_rate)])
    heart_rate_ts = np.array([np.nan, np.nan])  if len(heart_rate_ts) == 0 else heart_rate_ts
    feats = np.concatenate([feats, get_feature_list(heart_rate_ts)])
    
    heart_rate = np.array([np.nan, np.nan])  if len(heart_rate) == 1 else heart_rate
    feats = np.concatenate([feats, get_feature_list(np.diff(heart_rate))])
    heart_rate_ts = np.array([np.nan, np.nan])  if len(heart_rate_ts) == 1 else heart_rate_ts
    feats = np.concatenate([feats, get_feature_list(np.diff(heart_rate_ts))])
    
    
    return feats

def extract_feature_batch(X_signal_batch):
    X_feat = []
    for i in range(X_signal_batch.shape[0]):
        signal = X_signal_batch.iloc[i]
        signal_cut = np.array(signal.dropna())
        X_feat.append(extract_feature(signal_cut, 250))
    return np.array(X_feat)

In [3]:
X_train_data = pd.read_csv("../X_train.csv")
indices_train = np.array(X_train_data)[:,0]
X_train_data.drop("id", axis=1, inplace = True)

X_test_data =  pd.read_csv("../X_test.csv")
indices_test = np.array(X_test_data)[:,0]
X_test_data.drop("id", axis=1, inplace = True)


In [10]:
X_train = extract_feature_batch(X_train_data)
X_test = extract_feature_batch(X_test_data)

(65, 150)
(36, 150)
(30, 150)
(65, 150)
(45, 150)
(26, 150)
(38, 150)
(42, 150)
(60, 150)
(10, 150)
(27, 150)
(32, 150)
(25, 150)
(35, 150)
(31, 150)
(29, 150)
(19, 150)
(45, 150)
(27, 150)
(13, 150)
(35, 150)
(37, 150)
(30, 150)
(53, 150)
(35, 150)
(31, 150)
(25, 150)
(27, 150)
(26, 150)
(18, 150)
(48, 150)
(33, 150)
(35, 150)
(76, 150)
(43, 150)
(25, 150)
(39, 150)
(28, 150)
(85, 150)
(33, 150)
(52, 150)
(37, 150)
(54, 150)
(29, 150)
(55, 150)
(33, 150)
(43, 150)
(10, 150)
(44, 150)
(82, 150)
(37, 150)
(25, 150)
(29, 150)
(34, 150)
(24, 150)
(19, 150)
(46, 150)
(40, 150)
(31, 150)
(34, 150)
(34, 150)
(27, 150)
(33, 150)
(36, 150)
(53, 150)
(39, 150)
(34, 150)
(31, 150)
(31, 150)
(14, 150)
(33, 150)
(23, 150)
(44, 150)
(40, 150)
(38, 150)
(24, 150)
(36, 150)
(56, 150)
(50, 150)
(45, 150)
(28, 150)
(29, 150)
(54, 150)
(40, 150)
(31, 150)
(12, 150)
(38, 150)
(38, 150)
(30, 150)
(47, 150)
(32, 150)
(34, 150)
(70, 150)
(30, 150)
(29, 150)
(40, 150)
(19, 150)
(55, 150)
(39, 150)
(29, 150)


(30, 150)
(36, 150)
(46, 150)
(31, 150)
(34, 150)
(38, 150)
(39, 150)
(43, 150)
(30, 150)
(37, 150)
(27, 150)
(41, 150)
(66, 150)
(82, 150)
(38, 150)
(28, 150)
(40, 150)
(85, 150)
(33, 150)
(33, 150)
(59, 150)
(30, 150)
(34, 150)
(62, 150)
(19, 150)
(20, 150)
(30, 150)
(41, 150)
(59, 150)
(34, 150)
(31, 150)
(30, 150)
(39, 150)
(71, 150)
(31, 150)
(37, 150)
(29, 150)
(30, 150)
(28, 150)
(57, 150)
(29, 150)
(25, 150)
(35, 150)
(34, 150)
(31, 150)
(27, 150)
(28, 150)
(36, 150)
(91, 150)
(62, 150)
(57, 150)
(25, 150)
(31, 150)
(61, 150)
(11, 150)
(26, 150)
(34, 150)
(39, 150)
(33, 150)
(40, 150)
(33, 150)
(78, 150)
(18, 150)
(53, 150)
(26, 150)
(23, 150)
(34, 150)
(24, 150)
(86, 150)
(24, 150)
(41, 150)
(38, 150)
(62, 150)
(38, 150)
(25, 150)
(35, 150)
(38, 150)
(77, 150)
(47, 150)
(35, 150)
(36, 150)
(40, 150)
(11, 150)
(25, 150)
(26, 150)
(31, 150)
(26, 150)
(31, 150)
(36, 150)
(31, 150)
(50, 150)
(38, 150)
(29, 150)
(35, 150)
(16, 150)
(33, 150)
(18, 150)
(34, 150)
(32, 150)
(47, 150)


(70, 150)
(33, 150)
(27, 150)
(51, 150)
(44, 150)
(33, 150)
(36, 150)
(41, 150)
(51, 150)
(23, 150)
(22, 150)
(36, 150)
(69, 150)
(25, 150)
(84, 150)
(32, 150)
(25, 150)
(40, 150)
(35, 150)
(30, 150)
(45, 150)
(24, 150)
(43, 150)
(45, 150)
(27, 150)
(25, 150)
(39, 150)
(73, 150)
(27, 150)
(31, 150)
(30, 150)
(65, 150)
(32, 150)
(45, 150)
(27, 150)
(15, 150)
(21, 150)
(34, 150)
(56, 150)
(95, 150)
(31, 150)
(35, 150)
(37, 150)
(72, 150)
(33, 150)
(42, 150)
(70, 150)
(62, 150)
(35, 150)
(29, 150)
(74, 150)
(17, 150)
(28, 150)
(30, 150)
(14, 150)
(28, 150)
(43, 150)
(37, 150)
(62, 150)
(60, 150)
(65, 150)
(37, 150)
(26, 150)
(27, 150)
(27, 150)
(37, 150)
(27, 150)
(50, 150)
(32, 150)
(39, 150)
(30, 150)
(37, 150)
(29, 150)
(30, 150)
(31, 150)
(30, 150)
(29, 150)
(47, 150)
(43, 150)
(27, 150)
(31, 150)
(27, 150)
(29, 150)
(25, 150)
(34, 150)
(38, 150)
(43, 150)
(42, 150)
(47, 150)
(25, 150)
(40, 150)
(65, 150)
(25, 150)
(39, 150)
(14, 150)
(53, 150)
(27, 150)
(12, 150)
(62, 150)
(54, 150)


(36, 150)
(40, 150)
(29, 150)
(38, 150)
(32, 150)
(34, 150)
(35, 150)
(27, 150)
(29, 150)
(74, 150)
(29, 150)
(35, 150)
(65, 150)
(41, 150)
(16, 150)
(40, 150)
(26, 150)
(35, 150)
(19, 150)
(33, 150)
(29, 150)
(29, 150)
(25, 150)
(30, 150)
(34, 150)
(60, 150)
(70, 150)
(28, 150)
(25, 150)
(82, 150)
(37, 150)
(38, 150)
(43, 150)
(32, 150)
(39, 150)
(30, 150)
(36, 150)
(18, 150)
(43, 150)
(45, 150)
(41, 150)
(33, 150)
(33, 150)
(43, 150)
(29, 150)
(32, 150)
(31, 150)
(31, 150)
(26, 150)
(41, 150)
(37, 150)
(26, 150)
(29, 150)
(68, 150)
(56, 150)
(30, 150)
(19, 150)
(20, 150)
(36, 150)
(48, 150)
(30, 150)
(30, 150)
(39, 150)
(28, 150)
(40, 150)
(28, 150)
(74, 150)
(33, 150)
(71, 150)
(26, 150)
(33, 150)
(53, 150)
(29, 150)
(28, 150)
(17, 150)
(29, 150)
(30, 150)
(33, 150)
(25, 150)
(9, 150)
(41, 150)
(17, 150)
(33, 150)
(63, 150)
(34, 150)
(33, 150)
(39, 150)
(40, 150)
(66, 150)
(32, 150)
(24, 150)
(29, 150)
(21, 150)
(35, 150)
(36, 150)
(28, 150)
(31, 150)
(41, 150)
(27, 150)
(28, 150)
(

(50, 150)
(24, 150)
(33, 150)
(38, 150)
(36, 150)
(65, 150)
(34, 150)
(36, 150)
(25, 150)
(27, 150)
(58, 150)
(40, 150)
(30, 150)
(32, 150)
(26, 150)
(23, 150)
(33, 150)
(35, 150)
(43, 150)
(25, 150)
(42, 150)
(55, 150)
(43, 150)
(13, 150)
(70, 150)
(23, 150)
(33, 150)
(31, 150)
(34, 150)
(41, 150)
(34, 150)
(33, 150)
(53, 150)
(40, 150)
(34, 150)
(28, 150)
(55, 150)
(34, 150)
(35, 150)
(36, 150)
(14, 150)
(34, 150)
(47, 150)
(28, 150)
(52, 150)
(26, 150)
(30, 150)
(11, 150)
(31, 150)
(73, 150)
(35, 150)
(30, 150)
(32, 150)
(31, 150)
(26, 150)
(43, 150)
(28, 150)
(30, 150)
(11, 150)
(30, 150)
(45, 150)
(71, 150)
(34, 150)
(18, 150)
(24, 150)
(22, 150)
(29, 150)
(22, 150)
(29, 150)
(34, 150)
(17, 150)
(46, 150)
(31, 150)
(33, 150)
(41, 150)
(39, 150)
(33, 150)
(13, 150)
(35, 150)
(25, 150)
(29, 150)
(33, 150)
(34, 150)
(20, 150)
(86, 150)
(18, 150)
(29, 150)
(26, 150)
(36, 150)
(36, 150)
(54, 150)
(39, 150)
(26, 150)
(35, 150)
(34, 150)
(28, 150)
(37, 150)
(30, 150)
(47, 150)
(30, 150)


...

(91, 150)
(27, 150)
(28, 150)
(29, 150)
(28, 150)
(39, 150)
(35, 150)
(33, 150)
(30, 150)
(35, 150)
(25, 150)
(31, 150)
(37, 150)
(44, 150)
(61, 150)
(36, 150)
(34, 150)
(24, 150)
(11, 150)
(44, 150)
(34, 150)
(65, 150)
(88, 150)
(76, 150)
(32, 150)
(29, 150)
(44, 150)
(30, 150)
(15, 150)
(28, 150)
(47, 150)
(101, 150)
(87, 150)
(35, 150)
(39, 150)
(32, 150)
(27, 150)
(37, 150)
(21, 150)
(24, 150)
(58, 150)
(42, 150)
(60, 150)
(8, 150)
(19, 150)
(38, 150)
(32, 150)
(28, 150)
(33, 150)
(30, 150)
(41, 150)
(15, 150)
(30, 150)
(27, 150)
(62, 150)
(39, 150)
(88, 150)
(46, 150)
(20, 150)
(57, 150)
(72, 150)
(29, 150)
(37, 150)
(28, 150)
(25, 150)
(15, 150)
(22, 150)
(30, 150)
(39, 150)
(46, 150)
(25, 150)
(35, 150)
(11, 150)
(38, 150)
(35, 150)
(31, 150)
(41, 150)
(34, 150)
(52, 150)
(36, 150)
(54, 150)
(27, 150)
(24, 150)
(61, 150)
(27, 150)
(28, 150)
(48, 150)
(31, 150)
(31, 150)
(35, 150)
(55, 150)
(13, 150)
(40, 150)
(43, 150)
(32, 150)
(27, 150)
(31, 150)
(29, 150)
(37, 150)
(32, 150)


(70, 150)
(31, 150)
(38, 150)
(56, 150)
(17, 150)
(27, 150)
(22, 150)
(66, 150)
(14, 150)
(35, 150)
(90, 150)
(30, 150)
(30, 150)
(30, 150)
(64, 150)
(38, 150)
(27, 150)
(63, 150)
(43, 150)
(30, 150)
(32, 150)
(19, 150)
(31, 150)
(36, 150)
(25, 150)
(32, 150)
(27, 150)
(28, 150)
(26, 150)
(32, 150)
(33, 150)
(22, 150)
(24, 150)
(22, 150)
(28, 150)
(60, 150)
(32, 150)
(34, 150)
(45, 150)
(26, 150)
(39, 150)
(18, 150)
(25, 150)
(20, 150)
(38, 150)
(36, 150)
(36, 150)
(9, 150)
(49, 150)
(69, 150)
(37, 150)
(36, 150)
(54, 150)
(22, 150)
(36, 150)
(84, 150)
(73, 150)
(71, 150)
(27, 150)
(33, 150)
(61, 150)
(12, 150)
(30, 150)
(31, 150)
(45, 150)
(53, 150)
(30, 150)
(33, 150)
(25, 150)
(31, 150)
(58, 150)
(41, 150)
(40, 150)
(62, 150)
(41, 150)
(31, 150)
(34, 150)
(40, 150)
(32, 150)
(44, 150)
(33, 150)
(36, 150)
(67, 150)
(52, 150)
(31, 150)
(35, 150)
(34, 150)
(37, 150)
(60, 150)
(45, 150)
(35, 150)
(49, 150)
(22, 150)
(33, 150)
(34, 150)
(24, 150)
(20, 150)
(51, 150)
(42, 150)
(26, 150)
(

(28, 150)
(54, 150)
(40, 150)
(34, 150)
(34, 150)
(50, 150)
(38, 150)
(37, 150)
(33, 150)
(45, 150)
(31, 150)
(45, 150)
(78, 150)
(38, 150)
(31, 150)
(31, 150)
(40, 150)
(36, 150)
(40, 150)
(24, 150)
(35, 150)
(33, 150)
(37, 150)
(66, 150)
(43, 150)
(34, 150)
(68, 150)
(85, 150)
(29, 150)
(53, 150)
(54, 150)
(39, 150)
(56, 150)
(36, 150)
(39, 150)
(27, 150)
(36, 150)
(35, 150)
(13, 150)
(28, 150)
(55, 150)
(29, 150)
(38, 150)
(34, 150)
(27, 150)
(33, 150)
(37, 150)
(23, 150)
(38, 150)
(19, 150)
(29, 150)
(56, 150)
(36, 150)
(35, 150)
(28, 150)
(26, 150)
(45, 150)
(29, 150)
(14, 150)
(38, 150)
(24, 150)
(17, 150)
(30, 150)
(25, 150)
(27, 150)
(38, 150)
(71, 150)
(47, 150)
(32, 150)
(33, 150)
(34, 150)
(53, 150)
(61, 150)
(25, 150)
(64, 150)
(107, 150)
(29, 150)
(48, 150)
(69, 150)
(39, 150)
(29, 150)
(20, 150)
(32, 150)
(36, 150)
(21, 150)
(25, 150)
(16, 150)
(27, 150)
(32, 150)
(31, 150)
(31, 150)
(78, 150)
(52, 150)
(34, 150)
(90, 150)
(33, 150)
(40, 150)
(25, 150)
(42, 150)
(29, 150)

(35, 150)
(20, 150)
(17, 150)
(42, 150)
(88, 150)
(48, 150)
(65, 150)
(33, 150)
(40, 150)
(30, 150)
(28, 150)
(27, 150)
(34, 150)
(27, 150)
(28, 150)
(25, 150)
(31, 150)
(29, 150)
(38, 150)
(15, 150)
(27, 150)
(16, 150)
(32, 150)
(40, 150)
(13, 150)
(46, 150)
(41, 150)
(53, 150)
(91, 150)
(35, 150)
(31, 150)
(32, 150)
(20, 150)
(38, 150)
(42, 150)
(25, 150)
(40, 150)
(36, 150)
(33, 150)
(65, 150)
(34, 150)
(62, 150)
(28, 150)
(46, 150)
(23, 150)
(31, 150)
(41, 150)
(28, 150)
(52, 150)
(44, 150)
(33, 150)
(36, 150)
(33, 150)
(26, 150)
(39, 150)
(63, 150)
(32, 150)
(29, 150)
(14, 150)
(55, 150)
(99, 150)
(28, 150)
(18, 150)
(33, 150)
(36, 150)
(69, 150)
(21, 150)
(68, 150)
(33, 150)
(47, 150)
(40, 150)
(25, 150)
(45, 150)
(35, 150)
(58, 150)
(14, 150)
(30, 150)
(39, 150)
(31, 150)
(22, 150)
(32, 150)
(62, 150)
(53, 150)
(36, 150)
(89, 150)
(42, 150)
(73, 150)
(35, 150)
(26, 150)
(28, 150)
(32, 150)
(48, 150)
(63, 150)
(34, 150)
(34, 150)
(27, 150)
(41, 150)
(26, 150)
(31, 150)
(56, 150)


(38, 150)
(47, 150)
(63, 150)
(41, 150)
(65, 150)
(22, 150)
(32, 150)
(38, 150)
(27, 150)
(34, 150)
(38, 150)
(29, 150)
(33, 150)
(34, 150)
(35, 150)
(37, 150)
(23, 150)
(31, 150)
(22, 150)
(28, 150)
(37, 150)
(38, 150)
(34, 150)
(28, 150)
(31, 150)
(60, 150)
(36, 150)
(33, 150)
(40, 150)
(37, 150)
(49, 150)
(33, 150)
(59, 150)
(39, 150)
(28, 150)
(31, 150)
(43, 150)
(39, 150)
(81, 150)
(49, 150)
(41, 150)
(31, 150)
(34, 150)
(32, 150)
(36, 150)
(57, 150)
(31, 150)
(29, 150)
(28, 150)
(49, 150)
(37, 150)
(46, 150)
(30, 150)
(29, 150)
(34, 150)
(27, 150)
(32, 150)
(26, 150)
(35, 150)
(16, 150)
(41, 150)
(45, 150)
(28, 150)
(61, 150)
(26, 150)
(59, 150)
(23, 150)
(24, 150)
(22, 150)
(37, 150)
(47, 150)
(35, 150)
(27, 150)
(35, 150)
(29, 150)
(24, 150)
(51, 150)
(31, 150)
(20, 150)
(12, 150)
(32, 150)
(36, 150)
(40, 150)
(28, 150)
(11, 150)
(39, 150)
(32, 150)
(42, 150)
(26, 150)
(34, 150)
(26, 150)
(44, 150)
(38, 150)
(77, 150)
(12, 150)
(32, 150)
(31, 150)
(33, 150)
(42, 150)
(35, 150)


In [11]:
X_train_feat = np.insert(X_train,0,[indices_train],axis=1)
X_test_feat = np.insert(X_test,0,[indices_test],axis= 1)

In [12]:
df_X_train = pd.DataFrame(X_train_feat)
df_X_test = pd.DataFrame(X_test_feat)
df_X_train.to_csv("X_train_feature_rm.csv", index=False)
df_X_test.to_csv("X_test_feature_rm.csv", index=False)